# Setup

In [527]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango pdflatex

Note: you may need to restart the kernel to use updated packages.


In [337]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

Set up the OpenAI Assistant

In [338]:
from openai import OpenAI
client = OpenAI()

#  get the prompt from the file system
with open('prompt.md', 'r') as file:
    # Read the entire content of the file into a single string variable
    prompt = file.read()

# simple retrieve of the Assistant - not necessary because we are doing an update
# my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# update the assistant with the latest prompt
my_assistant = client.beta.assistants.update(
  "asst_3WQiw8VJ91XWikievImBqwrZ",
  instructions=prompt,
  name="Interview Playbook Generator",
  tools=[{"type": "retrieval"}],
  model="gpt-4-1106-preview"
  # model="gpt-3.5-turbo-1106"
)

In [339]:
# create a thread
my_thread = client.beta.threads.create()

import time

def create_run(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as John Gruber's markdown with no delimiters and with no commentary whatsoever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"


## Initial File Upload and Prompt

In [340]:
import time

# Which example to run
choice = 'medical-assistant'
#choice = 'infra-engineer'
#choice = 'customer-success-manager'
#choice = 'real-estate-litigator'

destination_directory = "examples/" + choice + "/"
html_destination = destination_directory + 'playbook.html'
pdf_destination = destination_directory + 'playbook.pdf'
markdown_destination = destination_directory + 'playbook.md'
word_destination = destination_directory + 'playbook.docx'

company_file = "examples/" + choice + "/about-company.md"

# Upload the company description
file_company_values = client.files.create(
  file=open(company_file, "rb"),
  purpose="assistants"
)

job_description_file = "examples/" + choice + "/job-description.md"
# Open the file and read the contents
with open(job_description_file, 'r') as file:
    # Read the entire content of the file into a single string variable
    job_description = file.read()

# Upload the job description file
file_job_description = client.files.create(
  file=open(job_description_file, "rb"),
  purpose="assistants"
)

# give OpenAI time to process the file
time.sleep(10)


# Priming the Pump

In [341]:
# Refresh the context to ensure it has read the files
primer_values = create_run(client,my_thread,my_assistant,"""
            Write a detailed description of the company values from the about company file, using the company name.
            """, [file_company_values.id])
print(primer_values)

# About Pacific Wellness Group - Academic Healthcare Consortium

Based on the search results, here is a summary of the key information about the mission, vision, and values of Pacific Wellness Group - Academic Healthcare Consortium:

## Mission:
- To be the premier community network for the Pacific Wellness Group System, delivering the best patient experience with the most effective clinical outcomes (per Academic Medicine Partners website).

## Vision:
- Healing humanity through science and compassion, one patient at a time.

## Values:
- Innovation - embracing innovation, change and continuous improvement to raise quality of care and shape the future of medicine 
- Connection - cultivating community, connection and care for each other to nurture trust, respect, belonging and shared purpose
- Teamwork - collaborating across roles, ranks, units and departments to unlock possibilities for greater impact and fulfillment
- Mentorship - nurturing and pursuing ongoing professional growth to

In [342]:
# Refresh the context to ensure it has read the files and create context. Not included in final doc.
primer_job = create_run(client,my_thread,my_assistant,"""
            Write the job title contained in the job description file.
            Then, create the list of 5 technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description. Create the list of 5 behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            For each competency, list 3 or 4 subcompetencies and their descriptions to enable more deep exploration.
            """, [file_job_description.id])
print(primer_job)


# Medical Assistant II

## Technical Competencies

### Clinical Task Proficiency
Expertise in performing daily clinical tasks such as checking vital signs, patient preparation, and assisting with medical procedures. 
- **Vital Signs**: Accurately measuring and recording patient vital signs.
- **Patient Preparation**: Effective patient preparation for examinations and procedures.
- **Procedure Assistance**: Assisting healthcare professionals during medical procedures.
- **Infection Control**: Maintaining high standards of hygiene and infection control practices.

### Medical Knowledge
Advanced understanding of medical terminologies, procedures, and patient care protocols as applicable in a clinical setting. 
- **Medical Terminology**: Proficient in medical vocabulary used in a clinical setting.
- **Patient Care Protocols**: Knowledgeable about patient care standards and protocols.
- **Procedural Knowledge**: Familiarity with common medical procedures done at the clinic.
- **Policy Adher

# Create Introduction

In [343]:
playbook_title = create_run(client,my_thread,my_assistant,"""
            Produce the title for the playbook with no markdown.
            """)
print(playbook_title)

Interview Playbook for Medical Assistant II




In [344]:
introduction = create_run(client,my_thread,my_assistant,"""
            Generate the introduction for the guide with no headings
            """)
print(intro)

This Interview Playbook is designed to equip the hiring team with a structured approach to evaluate candidates for the role of Department Attorney at Maxwell Fairview. The position of Department Attorney is a vital role within the company, requiring a blend of extensive legal expertise and strong interpersonal skills. The competencies outlined here align with the firm's values of unparalleled service, respect, and inclusion, and ensure the selection of a candidate who embodies these principles. The objectives of the interview include assessing technical skills, behavioral qualities, and overall fit with the company culture and values.




In [345]:
interview_structure = create_run(client,my_thread,my_assistant,"""
            Generate the Interview Structure with no headings. Include mention of the competencies in the interview structure.
            """)
print(interview_structure)

The interview for the Medical Assistant II position is structured to take place over a span of 45-60 minutes and is divided into several parts to assess both technical and behavioral competencies.

- The first 10 minutes are allocated for introductions and an overview of the role and company.
- This is followed by a 20-minute segment dedicated to evaluating the technical competencies: Clinical Task Proficiency, Medical Knowledge, EHR Management, Technical Equipment Handling, and BLS Procedures.
- The subsequent 20 minutes will focus on behavioral competencies such as Service Excellence, Team Collaboration, Communication Skills, Continuous Improvement, and Organizational Skills. During this time, the interviewers will explore the candidates' past experiences and their alignment with company values.
- The final 10-15 minutes are reserved for the candidate's questions and a wrap-up of the interview, providing a clear understanding of the subsequent steps in the hiring process.




# Create Evaluation Criteria

In [346]:
eval_criteria = create_run(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

## Evaluation Criteria

The evaluation criteria for the Medical Assistant II position are centered around a comprehensive understanding of both technical and behavioral competencies. Each competency will be assessed through the interview process, with careful attention to how the candidate's qualifications and character align with the company's mission and values.

| Competency                 | Assessment | Comments |
|----------------------------|------------|----------|
| Clinical Task Proficiency  |            |          |
| Medical Knowledge          |            |          |
| EHR Management             |            |          |
| Technical Equipment Handling |          |          |
| BLS Procedures             |            |          |
| Service Excellence         |            |          |
| Team Collaboration         |            |          |
| Communication Skills       |            |          |
| Continuous Improvement     |            |          |
| Organizational Skills    

# Create Technical Assessment

In [347]:
technical_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

For the Medical Assistant II role, the technical assessment focuses on evaluating the hands-on skills and medical knowledge that are essential to the job. The assessment will determine the candidate's capacity to perform clinical tasks effectively and efficiently, thereby ensuring the highest quality of patient care, in alignment with Pacific Wellness Group's mission and values.

### Technical Competence 1: Clinical Task Proficiency

Clinical Task Proficiency is crucial for a Medical Assistant II, as it involves direct patient interactions and a wide range of essential clinical tasks. This competency is fundamental to ensuring that all procedures are conducted safely and effectively, in line with the high standards of patient care expected at Pacific Wellness Group.

#### Lead Question

> **"Walk us through the process you would follow to prepare a patient for a minor surgical procedure and how you assist during the procedure. What steps do you take to ensure t

In [348]:
technical_assessment_2 = create_run(client,my_thread,my_assistant,"""
    Generate the 2nd Technical Assessment. Don't include the main section header.
    """)
print(technical_assessment_2)

### Technical Competence 2: Medical Knowledge

Deep familiarity with medical terminology, patient care protocols, and procedures informs a Medical Assistant II's ability to communicate effectively with healthcare professionals and deliver high-quality patient care, reflecting Pacific Wellness Group's commitment to excellence and continuous improvement.

#### Lead Question

> **"Can you describe a time when your medical knowledge significantly impacted the outcome of a patient’s care? How did you apply your understanding of patient care protocols to communicate effectively with other healthcare team members?"**

_Look for instances where comprehensive medical knowledge was applied in a practical, impactful way, demonstrating both understanding and the ability to collaborate for better patient outcomes._

#### Diving Deeper

- **Medical Terminology**: "Discuss how you ensure your use of medical terminology is both accurate and understandable to patients and team members."
- **Patient Car

In [349]:
technical_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Electronic Health Record (EHR) Management

Proficiency in EHR Management reflects a Medical Assistant II's ability to handle patient data with accuracy and care, a critical component of modern healthcare that aligns with Pacific Wellness Group’s values of innovation and participation.

#### Lead Question

> **"Describe your process for ensuring accuracy and security when entering patient data into an EHR system. Can you give an example where your attention to detail in EHR management made a difference in patient care or clinic operations?"**

_Look for a systematic approach to data management and examples of problem-solving that safeguard patient confidentiality and promote data integrity._

#### Diving Deeper

- **Data Entry**: "What steps do you take to verify the information before updating it into the patient's record?"
- **Patient Communication**: "How do you manage and prioritize patient message responses through the EHR portal?"
- **Information Securi

In [350]:
technical_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Technical Equipment Handling

The ability to handle technical equipment with proficiency underscores a Medical Assistant II's role in ensuring operational excellence and patient safety, in line with Pacific Wellness Group's commitment to innovation and quality care.

#### Lead Question

> **"In a clinical setting, you are often required to operate and maintain a range of technical equipment. Please describe the process you follow for the regular maintenance and operation of a piece of equipment you are familiar with."**

_Look for a detailed understanding of the equipment's operation and maintenance, as well as adherence to safety standards._

#### Diving Deeper

- **Equipment Operation**: "Please give examples of medical equipment you are experienced with and the training you underwent to operate them proficiently."
- **Maintenance**: "How do you ensure the equipment you are responsible for is always functioning properly and ready for use?"
- **Troubleshoot

In [351]:
technical_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: BLS Procedures

Mastery of Basic Life Support (BLS) procedures is a vital skill for a Medical Assistant II, aligned with Pacific Wellness Group's values of action and participation, to provide immediate care in life-threatening situations.

#### Lead Question

> **"Tell us about a time when you had to use your BLS training in a real-life scenario. What actions did you take, and how did you ensure the safety and well-being of the patient and others present?"**

_Look for examples of quick decision-making, practical application of BLS skills, and the impact of such actions on the patient's outcome._

#### Diving Deeper

- **CPR Certification**: "How do you keep your CPR certification current, and what additional steps do you take to ensure you're prepared for an emergency?"
- **Emergency Response**: "Explain the steps you would take if you were the first to respond to a patient in cardiac arrest in the clinic."
- **AED Usage**: "Describe your experience with u

In [352]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [353]:
behavioral_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Assessment

In assessing behavioral competencies for the Medical Assistant II role, we seek to understand how candidates exhibit their professional behaviors in various situations. This assessment helps in determining their alignment with Pacific Wellness Group's values and their potential to contribute positively to the working environment and patient care.

### Behavioral Competence 1: Service Excellence

Service Excellence is about providing exceptional care and service, reflecting the caring and patient-centered ethos of Pacific Wellness Group.

#### Lead Question

> **"Provide an example of a time when you went above and beyond for a patient. What was the situation, and why did you go the extra mile?"**

_Look for instances where the candidate demonstrates a genuine commitment to patient care and the ability to embody the company's C-I-CARE values._

#### Diving Deeper

- **Patient Interaction**: "Tell me about a time when you had to manage a difficult patient situat

In [354]:
behavioral_assessment_2 = create_run(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Team Collaboration

Effective Team Collaboration is essential to provide comprehensive care, aligning with Pacific Wellness Group's value of teamwork which emphasizes the importance of unity and cooperation among staff.

#### Lead Question

> **"Can you talk about a time when you had to collaborate with a team under challenging circumstances? What role did you play, and what was the outcome?"**

_Look for responses that reflect the candidate's ability to work cohesively with others, navigate difficulties, and contribute to positive team dynamics._

#### Diving Deeper

- **Cross-Departmental Coordination**: "Describe an instance where you had to coordinate with another department to solve a patient care issue."
- **Conflict Resolution**: "Explain your approach to resolving disagreements within the team."
- **Shared Learning**: "Provide an example of when you learned something valuable from a team member and how it improved your work."
- **Supportive Engageme

In [355]:
behavioral_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Communication Skills

Strong Communication Skills are vital for effective patient engagement and collaboration with healthcare teams, reflecting Pacific Wellness Group's commitment to providing a clear, compassionate, and professional exchange of information.

#### Lead Question

> **"Provide an example of a situation where effective communication was critical to the success of a patient's treatment plan. What was your approach and how did it affect the outcome?"**

_Look for instances that demonstrate the candidate’s ability to clearly articulate critical information and show sensitivity towards patients' and colleagues' needs._

#### Diving Deeper

- **Patient Communication**: "How do you tailor your communication to ensure that patients with different levels of health literacy can understand their care plan?"
- **Documentation Practice**: "Discuss the importance of documentation accuracy and how you maintain it in your day-to-day responsibilities."
- **I

In [356]:
behavioral_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Continuous Improvement

Continuous Improvement is about embracing changes and seeking ways to enhance personal and organizational growth, resonating with Pacific Wellness Group’s value of innovation.

#### Lead Question

> **"Describe a time when you identified an area for improvement within your role or team's processes. What initiative did you take to address it, and what was the result?"**

_Look for evidence of a proactive approach to identifying opportunities for improvement and the candidate's willingness to take initiative in alignment with the company's innovative spirit._

#### Diving Deeper

- **Skill Enhancement**: "How have you pursued additional training or education to improve your performance and patient care?"
- **Quality Improvement**: "Can you provide an example of a quality improvement project you were involved in and what contributions you made?"
- **Adaptability**: "Talk about a moment when you had to adapt to a significant change at wo

In [357]:
behavioral_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Organizational Skills

Organizational Skills are critical for managing the complexities of a clinical environment, ensuring efficient use of time and resources, and maintaining an exceptional level of patient care, supporting Pacific Wellness Group's high standards of operational excellence.

#### Lead Question

> **"How do you manage your tasks and responsibilities in a fast-paced environment to ensure that all patient needs are met without compromising care quality?"**

_Look for clear strategies for prioritization, multitasking, and maintaining composure under pressure._

#### Diving Deeper

- **Time Management**: "Discuss your approach to prioritizing urgent and non-urgent tasks throughout the day."
- **Attention to Detail**: "Describe a situation where your attention to detail prevented an error in patient care or clinic operations."
- **Resource Allocation**: "Explain how you optimize the use of resources at hand to ensure the best possible patient ca

In [358]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [359]:
practical_assignment = create_run(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The practical assignment for the Medical Assistant II candidate involves a hands-on simulation to evaluate their expertise in clinical tasks and patient interaction. The assignment will consist of a standardized patient scenario where the candidate demonstrates their competency in preparing a patient for a procedure, taking vital signs, and inputting data into an EHR system, all the while maintaining clear communication and adherence to safety protocols.

### Objectives:
- To assess the candidate's applied clinical skills and their ability to perform routine tasks under pressure.
- To evaluate the candidate's proficiency with EHR management and their consideration for data accuracy and security.
- To observe the candidate's communication skills and service excellence when interacting with patients.

### Evaluation Criteria:
- Accuracy: Ensuring that the vital signs are taken and recorded correctly.
- Efficiency: Completing all tasks within the given timeframe w

# Create Team Interaction

In [360]:
team_interaction = create_run(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Interaction

Incorporating team interaction into the interview process serves to evaluate the candidate's ability to integrate effectively into the existing clinic staff dynamics and contribute positively to Pacific Wellness Group's collaborative environment.

### Rationale:
Team interaction is key to observing the candidate's interpersonal skills, adaptability to the clinic culture, and potential for synergistic work relationships.

### Discussion Points:
- Candidate’s experience in working with interdisciplinary teams.
- Approach to conflict resolution and maintaining positive team dynamics.
- Techniques for effective cross-departmental coordination and communication.

### Activities:
- Candidate participates in a mock team meeting to discuss patient care planning.
- Informal meet-and-greet with potential colleagues, providing insight into the candidate's interpersonal communication style.
- Collaboration in a brief problem-solving exercise to gauge teamwork and creativity.



# Create Candidate Q&A

In [361]:
candidate_questions = create_run(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

Candidates often come prepared with questions about the role, team, and company. Below are possible questions from the candidate along with informed responses that accurately represent Pacific Wellness Group and its values.

1. **"What opportunities for professional development does Pacific Wellness Group offer?"**
   - At Pacific Wellness Group, we take pride in our employees' growth and offer various professional development programs, including continuing education, mentorship, and leadership training.

2. **"How does Pacific Wellness Group integrate innovation in its daily operations?"**
   - Innovation is one of our core values. We regularly implement new healthcare technologies, encourage our staff to participate in improvement initiatives, and foster a culture where innovative ideas are valued and explored.

3. **"Can you describe the teamwork dynamics within the clinic?"**
   - Our clinic operates on a foundation of mutual respect and collaboration. Teams acros

# Finalize the Playbook HTML

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [441]:
import markdown

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_playbook_title = markdown.markdown(playbook_title, extensions=['extra','mdx_truly_sane_lists'])
html_intro = markdown.markdown(intro, extensions=['extra','mdx_truly_sane_lists'])
html_interview_structure = markdown.markdown(interview_structure, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



## Build the HTML

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [448]:
# Open the file and read the contents
with open('styles.css', 'r') as file:
    # Read the entire content of the file into a single string variable
    styles = file.read()


html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")


# Open the HTML file and read its content
with open('template.html', 'r') as file:
    html_template = file.read()

# Replace the placeholders with the actual variables
html_content = html_template.format(
    styles=styles,
    html_playbook_title=html_playbook_title,
    publication_date=publication_date,
    html_intro=html_intro,
    html_interview_structure=html_interview_structure,
    html_job_description=html_job_description,
    html_eval_criteria=html_eval_criteria,
    html_technical_assessment_1=html_technical_assessment_1,
    html_short_notes=html_short_notes,
    html_technical_assessment_2=html_technical_assessment_2,
    html_notes=html_notes,
    html_technical_assessment_3=html_technical_assessment_3,
    html_technical_assessment_4=html_technical_assessment_4,
    html_technical_assessment_5=html_technical_assessment_5,
    html_behavioral_assessment_1=html_behavioral_assessment_1,
    html_behavioral_assessment_2=html_behavioral_assessment_2,
    html_behavioral_assessment_3=html_behavioral_assessment_3,
    html_behavioral_assessment_4=html_behavioral_assessment_4,
    html_behavioral_assessment_5=html_behavioral_assessment_5,
    html_practical_assignment=html_practical_assignment,
    html_team_interaction=html_team_interaction,
    html_candidate_questions=html_candidate_questions
)

# Write HTML string to a file
with open(html_destination, 'w') as file:
    file.write(html_content)


# Display the Markdown-formatted text in the notebook
# from IPython.display import display, HTML
# display(HTML(html_content))


# Create the Documents

### Create PDF from HTML

In [449]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML(html_destination).write_pdf(pdf_destination, stylesheets=['styles.css'], full_fonts=True, hinting=True, presentational_hints=True)



## Create a markdown version

For use in GitHub as an example

In [553]:
notes_snippet_md = """##### Notes\n
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________\n
Rating (1-5): \\__________\n\n\n"""

from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time in a nice readable format
formatted_now = now.strftime("%B %d, %Y %-I:%M %p")

print(formatted_now)

playbook_md = (
            "---\n" + 
            "title: " + playbook_title  + "\n\n" +
            "author: Created by [Deliverables.ai](https://github.com/androb/deliverables)\n" +
            "date: " + formatted_now + "\n"
            "---\n" + 
            "## Introduction" + "\n\n" +
            introduction + "\n\n" +
            "### Interview Structure" + "\n\n" +
            interview_structure + "\n\n" +
            "### Job Description" + "\n\n" +
            job_description + "\n\n" +
            eval_criteria + "\n\n" +
            technical_assessment_1 + "\n\n" +
            notes_snippet_md +
            technical_assessment_2 + "\n\n" +
            notes_snippet_md +
            technical_assessment_3 + "\n\n" +
            notes_snippet_md +
            technical_assessment_4 + "\n\n" +
            notes_snippet_md +
            technical_assessment_5 + "\n\n" +
            notes_snippet_md +
            behavioral_assessment_1 + "\n\n" +
            notes_snippet_md +
            behavioral_assessment_2 + "\n\n" +
            notes_snippet_md +
            behavioral_assessment_3 + "\n\n" +
            notes_snippet_md +
            behavioral_assessment_4 + "\n\n" +
            notes_snippet_md +
            behavioral_assessment_5 + "\n\n" +
            notes_snippet_md +
            practical_assignment + "\n\n" +
            notes_snippet_md +
            team_interaction + "\n\n" +
            notes_snippet_md +
            candidate_questions + "\n\n" )

# Write playbook to a text file
with open(markdown_destination, 'w') as file:
    file.write(playbook_md)

December 26, 2023 2:02 PM


### Create Word Document from Markdown

In [569]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="custom-reference.docx" --toc=True --shift-heading-level-by -1
!pandoc -s {markdown_destination} -o {html_destination} --template=html-reference --toc=True


# use this extension to shift headings to differ between HTML and docx
# --shift-heading-level-by -1

# Convert from markdown to PDF via HTML
# !pandoc --pdf-engine "weasyprint" -s {markdown_destination} -o {pdf_destination} --reference-doc="custom-reference.docx" --toc=True
# !pandoc --pdf-engine "weasyprint" -s {word_destination} -o {pdf_destination}


In [542]:
pptx_destination = destination_directory + "playbook.pptx"
!pandoc -s {markdown_destination} -o {pptx_destination}